In [12]:
from transformers import pipeline
from tabulate import tabulate

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
results = [
    classifier("This is a guide about accounting for inventory in business operations.", candidate_labels=["accounting", "inventory management", "finance"]),
    classifier("The process of billing customers for services rendered.", candidate_labels=["billing", "accounts receivable", "finance"]),
    classifier("Accounts payable and accounts receivable are essential components of financial operations.", candidate_labels=["accounts payable", "accounts receivable", "finance"])
]

generator = pipeline("text-generation", model="distilgpt2")
generated_texts = [
    generator("In this accounting module, we will cover the invoicing process and best practices for", max_length=30, num_return_sequences=2),
    generator("In this accounting tutorial, we will learn about managing accounts payable and receivable, focusing on", max_length=30, num_return_sequences=2),
    generator("In this course, we will explore inventory accounting and cost management methods", max_length=30)
]

unmasker = pipeline("fill-mask", model="bert-base-uncased")
unmask_results = unmasker("Inventory accounting includes methods such as [MASK] inventory, weighted average, and FIFO.", top_k=2)

ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)
ner_results = ner("My name is Sarah and I work at Hugging Face in Brooklyn.")

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
qa_results = [
    qa_pipeline(question="What is invoicing?", context="Invoicing is a critical part of the billing process in accounting, where a bill is issued to a customer for services or goods."),
    qa_pipeline(question="What is accounts payable?", context="Accounts payable refers to money owed by a business to its suppliers, while accounts receivable represents the amount owed to the business by its customers.")
]

sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
sentiment_results = sentiment_classifier("The invoicing software is very user-friendly and efficient for tracking payments.")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary_result = summarizer(
    """
    Inventory accounting is a complex process that involves tracking the value of goods
    held by a business. Various methods are used, including FIFO, LIFO, and weighted
    average, each with its own impact on financial statements and tax obligations.
    Inventory accounting affects both the cost of goods sold and the net income reported
    by a company. Proper management of inventory accounting helps businesses make
    informed financial decisions, optimize stock levels, and maintain accurate financial records.
    """,
    max_length=60,
    min_length=30,
    do_sample=False
)

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-id-en")
text_to_translate = "Piutang usaha adalah jumlah yang harus dibayar oleh pelanggan kepada bisnis untuk barang atau layanan yang diberikan."
translation_result = translator(text_to_translate)

print("\nZero-shot Classification Results:")
zero_shot_results = [[result['sequence'], result['labels'], result['scores']] for result in results]
print(tabulate(zero_shot_results, headers=["Text", "Labels", "Scores"], tablefmt="grid"))

print("\nGenerated Texts:")
generated_texts_flat = [[text['generated_text']] for seq in generated_texts for text in seq]
print(tabulate(generated_texts_flat, headers=["Generated Text"], tablefmt="grid"))

print("\nUnmask Results:")
unmask_flat = [[unmask_result['sequence'], unmask_result['token_str'], unmask_result['score']] for unmask_result in unmask_results]
print(tabulate(unmask_flat, headers=["Masked Sentence", "Predicted Words", "Score"], tablefmt="grid"))

print("\nNER Results:")
ner_flat = [[entity['word'], entity['entity_group'], entity['score']] for entity in ner_results]
print(tabulate(ner_flat, headers=["Entity", "Type", "Score"], tablefmt="grid"))

print("\nQuestion-Answering Results:")
qa_flat = [[qa['score'], qa['answer']] for qa in qa_results]
print(tabulate(qa_flat, headers=["Score", "Answer"], tablefmt="grid"))

print("\nSentiment Analysis Result:")
sentiment_flat = [[result['label'], result['score']] for result in sentiment_results]
print(tabulate(sentiment_flat, headers=["Label", "Score"], tablefmt="grid"))

print("\nSummary Result:")
print(tabulate([[summary_result[0]['summary_text']]], headers=["Summary"], tablefmt="grid"))

print("\nTranslation Result:")
print(tabulate([[translation_result[0]['translation_text']]], headers=["Translated Text"], tablefmt="grid"))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassifica


Zero-shot Classification Results:
+--------------------------------------------------------------------------------------------+--------------------------------------------------------+-----------------------------------------------------------------+
| Text                                                                                       | Labels                                                 | Scores                                                          |
+============================================================================================+========================================================+=================================================================+
| This is a guide about accounting for inventory in business operations.                     | ['accounting', 'inventory management', 'finance']      | [0.5851579904556274, 0.39887627959251404, 0.015965737402439117] |
+--------------------------------------------------------------------------------------------